In [2]:
from rapidfuzz import process, fuzz, utils
import pandas as pd
from tqdm import tqdm

shs100k = pd.read_parquet("../data/shs100k2_yt.parquet")
datacos = pd.read_parquet("../data/datacos_yt.parquet")
shs_yt = pd.read_parquet("../data/shs_yt_yt.parquet")


# Rapid Fuzz Random Example

In [35]:
random_video = shs100k.sample(n=1)
random_video.head()

song_title = random_video.title.item().lower()
video_title = random_video.video_title.item().lower()
video_description = random_video.description.item().lower()

# song title in video title
st_vt = fuzz.token_set_ratio(song_title, video_title)
vt_st = fuzz.token_set_ratio(video_title, song_title)
# song title in description
st_vd = fuzz.token_set_ratio(song_title, video_description)
vd_st = fuzz.token_set_ratio(video_description, song_title)

print(f"song title: {song_title}")
print(f"video title: {video_title}")
print(f"video description: {video_description}")

print(f"st-vt {st_vt}")
print(f"vt-st {vt_st}")
print(f"vd-st {st_vd}")
print(f"st-vd {vd_st}")




song title: spanish is the loving tongue
video title: crew cuts - spanish is a loving tongue
video description: this was released on vee jay 569 in 1963, it's the flip of the three bells.  this was their only vee jay release, i'm not aware of any vee jay lp. i have never seen a stock copy of this record, but have heard it played on the radio a few times.  i hope you like it, feel free to comment.
st-vt 92.3076923076923
vt-st 92.3076923076923
vd-st 19.354838709677423
st-vd 19.354838709677423


# ChatGPT tries...gen NER dataset

In [40]:
import spacy

def create_ner_dataset(longer_sequence, shorter_sequence):
    # Load spaCy English model for tokenization
    nlp = spacy.load("en_core_web_sm")

    # Tokenize the longer and shorter sequences
    longer_tokens = [token.text for token in nlp(longer_sequence)]
    shorter_tokens = [token.text for token in nlp(shorter_sequence)]

    # Initialize the output dictionary with all tokens set to 0
    output_dict = {token: 0 for token in longer_tokens}

    # Find the start index of the overlapping region
    start_index = longer_tokens.index(shorter_tokens[0])

    # Mark the tokens in the overlapping region as 1
    for i in range(start_index, start_index + len(shorter_tokens)):
        output_dict[longer_tokens[i]] = 1

    return output_dict

# Example usage
longer_sequence = "This is a longer sequence with more tokens."
shorter_sequence = "longer sequence"
result = create_ner_dataset(longer_sequence, shorter_sequence)
print(result)


{'This': 0, 'is': 0, 'a': 0, 'longer': 1, 'sequence': 1, 'with': 0, 'more': 0, 'tokens': 0, '.': 0}


In [46]:
import pandas as pd
import spacy

def create_ner_dataset(row):
    # Tokenize the longer and shorter sequences
    longer_tokens = [(token.text, 'UNKN') for token in nlp(row['video_title'])]
    title_tokens = [(token.text, 'TITLE') for token in nlp(row['title'])]
    performer_tokens = [(token.text, 'PERFORMER') for token in nlp(row['performer'])]

    # Create sets of tokens for 'title' and 'performer'
    title_set = set(token for token, _ in title_tokens)
    performer_set = set(token for token, _ in performer_tokens)

    # Initialize the output list of tuples
    output_list = []

    # Populate the output list with tuples from 'title' and 'performer'
    for i, (token, _) in enumerate(longer_tokens):
        if token in title_set:
            output_list.append((i, token, 'TITLE'))
        elif token in performer_set:
            output_list.append((i, token, 'PERFORMER'))
        else:
            output_list.append((i, token, 'UNKN'))

    return output_list

# Example DataFrame
data = {'video_title': ["This is a longer sequence with more tokens.", "Another video title"],
        'title': ["longer sequence", "Another title"],
        'performer': ["more tokens", "Another performer"]}

df = pd.DataFrame(data)

# Load spaCy English model for tokenization
nlp = spacy.load("en_core_web_sm")

# Apply the create_ner_dataset function to each row of the DataFrame
result_list = df.apply(create_ner_dataset, axis=1).tolist()

# Print the list of lists of tuples
print(result_list)


[[(0, 'This', 'UNKN'), (1, 'is', 'UNKN'), (2, 'a', 'UNKN'), (3, 'longer', 'TITLE'), (4, 'sequence', 'TITLE'), (5, 'with', 'UNKN'), (6, 'more', 'PERFORMER'), (7, 'tokens', 'PERFORMER'), (8, '.', 'UNKN')], [(0, 'Another', 'TITLE'), (1, 'video', 'UNKN'), (2, 'title', 'TITLE')]]


In [1]:
def ner_row(row, col="title"):
    
    def encode(sub, sent):
        subwords, sentwords = sub.lower().split(), sent.lower().split()
        print(subwords)
        res = ["UNK" for _ in sentwords]    
        for i, word in enumerate(sentwords[:-len(subwords) + 1]):
            if all(x == y for x, y in zip(subwords, sentwords[i:i + len(subwords)])):
                for j in range(len(subwords)):
                    res[i + j] = col.upper()
        return res
    
    doc = row.video_title
    qry = row[col]
    
    seq_qry = encode(qry, doc)

    return seq_qry


df = shs100k.head()
df[["video_title", "title", "performer"]]


NameError: name 'shs100k' is not defined

In [80]:
df.apply(lambda x: ner_row(x, 'title'), axis=1)


['yesterday']
['yesterday']
['yesterday']
['yesterday']
['yesterday']


0    [UNK, UNK, UNK, UNK, UNK, UNK]
1              [UNK, UNK, UNK, UNK]
4              [UNK, UNK, UNK, UNK]
5              [UNK, UNK, UNK, UNK]
6              [UNK, UNK, UNK, UNK]
dtype: object

In [79]:
def encode(sub, sent):
    subwords, sentwords = sub.split(), sent.split()
    res = [0 for _ in sentwords]    
    for i, word in enumerate(sentwords[:-len(subwords) + 1]):
        if all(x == y for x, y in zip(subwords, sentwords[i:i + len(subwords)])):
            for j in range(len(subwords)):
                res[i + j] = 1
    return res

def encode2(sub, sent):
    subword, sentwords = sub, sent.split()
    res = [0] * len(sentwords)

    for i, word in enumerate(sentwords):
        if word == subword:
            res[i] = 1

    return res

encode2("yesterday i", "marianne faithful yesterday i")

[0, 0, 0, 0]

In [33]:
q="You're All I Need to Get By"
d="DIANA ROSS  you're all i need to get by"

fuzz.QRatio(q.lower(), d.lower())

81.81818181818181

In [44]:
import spacy

NER = spacy.load("en_core_web_sm")

doc = NER(d)
entities = [(e.label_,e.text) for e in doc.ents]



In [45]:
entities

[('PERSON', 'DIANA ROSS')]

In [6]:
# query (title, performer)
# yt attr (video title, channel name, description)
match_dict = {}

for query_column in ["title", "performer"]:
    
    print(f"qry col: {query_column}")
    match_dict[query_column] = {}
    
    for document_column in tqdm(["video_title", "description", "channel_name"]):
        
        print(f"doc col: {document_column}")

        match_dict[query_column][document_column] = {}
        
                        
        results = process.cdist(queries=shs100k[query_column].to_list(), 
                          choices=shs100k[document_column].to_list(), )
                        
        match_dict[query_column][document_column]["token_set_ratio"] = results
            


qry col: title
doc col: video_title


  0%|          | 0/11 [00:22<?, ?it/s]


KeyboardInterrupt: 

,title,video_title
30125,Manha de Carnaval,Mason Williams


In [ ]:
shs100k2